In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from os import path
from pathlib import Path  

import nltk
from nltk import sent_tokenize, word_tokenize

In [2]:
conditions_file = path.join('..','data','raw_data','/home/amy/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/2_Condition_Extraction/conditions.xlsx') # Enter metadata filename here
conditions = pd.read_excel(conditions_file);

In [3]:
conditions

,Unnamed: 0,Title,Source name 1,Description 1,Label 1,Source name 2,Description 2,Label 2
0,GSM5197020,"WT (stationary phase, OD 30) vs. WT (exponenti...","WT (stationary phase, OD 30)",genotype: wild type condition: WT harvested du...,Cy5,"WT (exponential Phase, OD 4)",genotype: wild type condition: WT harvested du...,Cy3
1,GSM5197021,"WT (stationary phase, OD 30) vs. WT (exponenti...","WT (stationary phase, OD 30)",genotype: wild type condition: WT harvested du...,Cy5,"WT (exponential Phase, OD 4)",genotype: wild type condition: WT harvested du...,Cy3
2,GSM5197022,"WT (stationary phase, OD 30) vs. WT (exponenti...","WT (stationary phase, OD 30)",genotype: wild type condition: WT harvested du...,Cy3,"WT (exponential Phase, OD 4)",genotype: wild type condition: WT harvested du...,Cy5
3,GSM5197023,"WT (stationary phase, OD 30) vs. WT (exponenti...","WT (stationary phase, OD 30)",genotype: wild type condition: WT harvested du...,Cy5,"WT (exponential Phase, OD 4)",genotype: wild type condition: WT harvested du...,Cy3
4,GSM5197024,"WT (stationary phase, OD 75) vs. WT (exponenti...","WT (stationary phase, OD 75)",genotype: wild type condition: WT harvested du...,Cy5,"WT (exponential Phase, OD 5)",genotype: wild type condition: WT harvested du...,Cy3
...,...,...,...,...,...,...,...,...
922,GSM5197943,WT PctaD::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-...,WT PctaD::Ptuf PctaE::Ptuf PctaC::Ptuf,genotype: promoter exchange condition: WT Pcta...,Cy3,WT,genotype: wild type condition: WT harvested du...,Cy5
923,GSM5197944,WT PctaD::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-...,WT PctaD::Ptuf PctaE::Ptuf PctaC::Ptuf,genotype: promoter exchange condition: WT Pcta...,Cy5,WT,genotype: wild type condition: WT harvested du...,Cy3
924,GSM5197945,WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-...,WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf,genotype: promoter exchange condition: WT Pnrd...,Cy5,WT,genotype: wild type condition: WT harvested du...,Cy3
925,GSM5197946,WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf vs. WT-...,WT PnrdH::Ptuf PctaE::Ptuf PctaC::Ptuf,genotype: promoter exchange condition: WT Pnrd...,Cy5,WT,genotype: wild type condition: WT harvested du...,Cy3


In [4]:
conditions_1 = conditions['Description 1'].tolist()

In [5]:
def parse_conditions(description):
    
    # find strain type
    i = 2
    temp = []
    while i < len(description) and description[i] != "condition":
        temp.append(description[i])
        i = i+1
    strain_type = ' '.join(temp)
    
    # find strain info
    i = i+2
    temp = []
    while i < len(description) and description[i] != "harvested" and description[i] != "harvest":
        temp.append(description[i])
        i = i+1
    strain_info = ' '.join(temp)
    
    # find growth info
    i = i+1
    temp = []
    while i < len(description) and description[i] != ",":
        temp.append(description[i])
        i = i+1
    growth_info = ' '.join(temp)
    
    # Bioreactor
    i = i+1
    bioreactor = False
    if description[i][0] == 'B' and description[i][1] == "i":
        bioreactor = True
        i = i+2
    
    # OD
    OD = "N/A"
    if description[i][0] == 'O':
        OD = description[i+1]
        i = i+3
    
    # Media
    temp = []
    while i < len(description) and description[i] != ",":
        temp.append(description[i])
        i = i+1
    media = ' '.join(temp)
    
    # Additional Information 
    i = i+1
    additional_info = []
    temp = []
    while i < len(description) and description[i] != "date":
        if description[i] == ',':
            temp_info = ' '.join(temp)
            additional_info.append(temp_info)
            temp = []
            i = i+1
        else:
            temp.append(description[i])
            i = i+1
    addition = ' '.join(additional_info)
    
    # Experimental Date
    Date = description[i+4]


    
    return strain_type, strain_info,growth_info,bioreactor,OD,media,addition,Date
        

In [6]:
strain_type = []
strain_info = []
growth_info = []
bioreactor = []
OD = []
media = []
date = []
additional_info = []
for j in range(len(conditions_1)):
    temp = word_tokenize(conditions_1[j])
    st, si, gi, b, o, m, ai, d = parse_conditions(temp)
    strain_type.append(st)
    strain_info.append(si)
    growth_info.append(gi)
    bioreactor.append(b)
    OD.append(o)
    media.append(m)
    date.append(d)
    additional_info.append(ai)

In [7]:
Serial_number = conditions['Unnamed: 0'].tolist()
CE_1 = pd.DataFrame(Serial_number, columns=['serial number'])
CE_1["strain type"] = strain_type 
CE_1["strain info"] = strain_info
CE_1["growth info"] = growth_info
CE_1["OD"] = OD
CE_1["bioreactor"] = bioreactor
CE_1["media"] = media
CE_1["date"] = date
CE_1["additioanl info"] = additional_info
CE_1

,serial number,strain type,strain info,growth info,OD,bioreactor,media,date,additioanl info
0,GSM5197020,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
1,GSM5197021,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
2,GSM5197022,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
3,GSM5197023,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
4,GSM5197024,wild type,WT,during stationary growth phase,75,False,CGXII glucose,2009-06,
...,...,...,...,...,...,...,...,...,...
922,GSM5197943,promoter exchange,WT PctaD : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2011-05/2012-05,
923,GSM5197944,promoter exchange,WT PctaD : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2011-05/2012-05,
924,GSM5197945,promoter exchange,WT PnrdH : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2012-02,
925,GSM5197946,promoter exchange,WT PnrdH : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2012-02,


In [9]:
filepath = Path('/home/amy/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/2_Condition_Extraction/conditions_source1_extracted.xlsx')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
CE_1.to_excel(filepath) 

In [10]:
conditions_2 = conditions['Description 2'].tolist()
strain_type = []
strain_info = []
growth_info = []
bioreactor = []
OD = []
media = []
date = []
additional_info = []
for j in range(len(conditions_2)):
    temp = word_tokenize(conditions_1[j])
    st, si, gi, b, o, m, ai, d = parse_conditions(temp)
    strain_type.append(st)
    strain_info.append(si)
    growth_info.append(gi)
    bioreactor.append(b)
    OD.append(o)
    media.append(m)
    date.append(d)
    additional_info.append(ai)

In [11]:
Serial_number = conditions['Unnamed: 0'].tolist()
CE_2 = pd.DataFrame(Serial_number, columns=['serial number'])
CE_2["strain type"] = strain_type 
CE_2["strain info"] = strain_info
CE_2["growth info"] = growth_info
CE_2["OD"] = OD
CE_2["bioreactor"] = bioreactor
CE_2["media"] = media
CE_2["date"] = date
CE_2["additioanl info"] = additional_info
CE_2

,serial number,strain type,strain info,growth info,OD,bioreactor,media,date,additioanl info
0,GSM5197020,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
1,GSM5197021,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
2,GSM5197022,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
3,GSM5197023,wild type,WT,during stationary growth phase,30,False,CGXII glucose,2008-06,
4,GSM5197024,wild type,WT,during stationary growth phase,75,False,CGXII glucose,2009-06,
...,...,...,...,...,...,...,...,...,...
922,GSM5197943,promoter exchange,WT PctaD : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2011-05/2012-05,
923,GSM5197944,promoter exchange,WT PctaD : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2011-05/2012-05,
924,GSM5197945,promoter exchange,WT PnrdH : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2012-02,
925,GSM5197946,promoter exchange,WT PnrdH : :Ptuf PctaE : :Ptuf PctaC : :Ptuf,during exponential growth phase,N/A,False,CGXII glucose,2012-02,


In [12]:
filepath = Path('/home/amy/Documents/GitHub/modulome-C_Glutamicum_Microarray_clean/2_Condition_Extraction/conditions_source2_extracted.xlsx')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
CE_2.to_excel(filepath) 